<h2 align='center' style='color:purple'>Dropout Regularization In Deep Neural Network</h2>

This is a dataset that describes sonar chirp returns bouncing off different services. The 60 input variables are the strength of the returns at different angles. It is a **binary classification problem** that requires a model to differentiate rocks from metal cylinders.

Dataset information: https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)
Download it from here: https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("./sonar_dataset.csv", header=None)
df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
94,0.0025,0.0309,0.0171,0.0228,0.0434,0.1224,0.1947,0.1661,0.1368,0.1430,...,0.0149,0.0077,0.0036,0.0114,0.0085,0.0101,0.0016,0.0028,0.0014,R
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
74,0.0109,0.0093,0.0121,0.0378,0.0679,0.0863,0.1004,0.0664,0.0941,0.1036,...,0.0077,0.0023,0.0117,0.0053,0.0077,0.0076,0.0056,0.0055,0.0039,R
192,0.0056,0.0267,0.0221,0.0561,0.0936,0.1146,0.0706,0.0996,0.1673,0.1859,...,0.0072,0.0055,0.0074,0.0068,0.0084,0.0037,0.0024,0.0034,0.0007,M
17,0.0192,0.0607,0.0378,0.0774,0.1388,0.0809,0.0568,0.0219,0.1037,0.1186,...,0.0331,0.0131,0.0120,0.0108,0.0024,0.0045,0.0037,0.0112,0.0075,R


In [4]:
df.shape

(208, 61)

In [5]:
# check for nan values
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [6]:
df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
           dtype='int64')

In [7]:
df[60].value_counts() # label is not skewed

M    111
R     97
Name: 60, dtype: int64

In [8]:
X = df.drop(60, axis=1)
y = df[60]
y.head()

0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object

In [9]:
y = pd.get_dummies(y, drop_first=True)
y.sample(5) # R --> 1 and M --> 0

,R
139,0
182,0
59,1
12,1
13,1


In [10]:
y.value_counts()

R
0    111
1     97
dtype: int64

In [11]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [13]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
67,0.0368,0.0403,0.0317,0.0293,0.0820,0.1342,0.1161,0.0663,0.0155,0.0506,...,0.0058,0.0091,0.0160,0.0160,0.0081,0.0070,0.0135,0.0067,0.0078,0.0068
14,0.0124,0.0433,0.0604,0.0449,0.0597,0.0355,0.0531,0.0343,0.1052,0.2120,...,0.0078,0.0083,0.0057,0.0174,0.0188,0.0054,0.0114,0.0196,0.0147,0.0062
164,0.0163,0.0198,0.0202,0.0386,0.0752,0.1444,0.1487,0.1484,0.2442,0.2822,...,0.0027,0.0077,0.0026,0.0031,0.0083,0.0020,0.0084,0.0108,0.0083,0.0033
179,0.0394,0.0420,0.0446,0.0551,0.0597,0.1416,0.0956,0.0802,0.1618,0.2558,...,0.0118,0.0146,0.0040,0.0114,0.0032,0.0062,0.0101,0.0068,0.0053,0.0087
19,0.0126,0.0149,0.0641,0.1732,0.2565,0.2559,0.2947,0.4110,0.4983,0.5920,...,0.0153,0.0092,0.0035,0.0098,0.0121,0.0006,0.0181,0.0094,0.0116,0.0063


# Using Deep Learning Model

## Model without Dropout Layer

In [14]:
import tensorflow as tf
from tensorflow import keras

In [15]:
model = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 1ms/step - loss: 0.6748 - accuracy: 0.6072
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6548 - accuracy: 0.6631
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6330 - accuracy: 0.6727
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6302 - accuracy: 0.6956
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5687 - accuracy: 0.7445
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5298 - accuracy: 0.7580
Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4940 - accuracy: 0.7827
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4748 - accuracy: 0.7701
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4512 - accuracy: 0.8171
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3867 - accuracy: 0.8276
Epoch 11/

20/20 [==============================] - 0s 1ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 83/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 84/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0075 - accuracy: 1.0000
Epoch 85/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 86/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 87/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 88/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 89/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 90/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 91/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0061 - accuracy: 1.0000
Epoch 92/100


In [16]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 1.0893 - accuracy: 0.7500


[1.0893199443817139, 0.75]

Training Accuracy >>> Test Accuracy

In [17]:
y_pred = model.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

[1.7989031e-08 9.9569345e-01 7.3072499e-01 2.1199130e-06 9.9999678e-01
 9.9985790e-01 8.5727727e-01 9.9999952e-01 2.2480404e-05 9.9999869e-01]
[0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]


In [18]:
y_test[:10]

,R
186,0
155,0
165,0
200,0
58,1
34,1
151,0
18,1
202,0
62,1


In [19]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.81      0.77        27
           1       0.77      0.68      0.72        25

    accuracy                           0.75        52
   macro avg       0.75      0.75      0.75        52
weighted avg       0.75      0.75      0.75        52



### Model with Dropout Layer

In [20]:
modeld = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

modeld.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modeld.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 1ms/step - loss: 0.7461 - accuracy: 0.4833
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6716 - accuracy: 0.5876
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7143 - accuracy: 0.4841
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7025 - accuracy: 0.4431
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7075 - accuracy: 0.4924
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7139 - accuracy: 0.4644
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6622 - accuracy: 0.5686
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6740 - accuracy: 0.6041
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6921 - accuracy: 0.4944
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6668 - accuracy: 0.5854
Epoch 11/

Epoch 83/100
20/20 [==============================] - 0s 1ms/step - loss: 0.2880 - accuracy: 0.8697
Epoch 84/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3252 - accuracy: 0.8729
Epoch 85/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3362 - accuracy: 0.8978
Epoch 86/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3081 - accuracy: 0.9299
Epoch 87/100
20/20 [==============================] - 0s 1ms/step - loss: 0.2994 - accuracy: 0.8763
Epoch 88/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4363 - accuracy: 0.7944
Epoch 89/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4068 - accuracy: 0.7992
Epoch 90/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3184 - accuracy: 0.9064
Epoch 91/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3056 - accuracy: 0.8500
Epoch 92/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3670 - accuracy: 0.8836


In [21]:
modeld.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.4162 - accuracy: 0.8077


[0.41619470715522766, 0.807692289352417]

Training Accuracy is still good but Test Accuracy Improved

In [22]:
y_pred = modeld.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

[1.0333454e-04 7.9011875e-01 9.0382969e-01 1.4036834e-02 9.9719894e-01
 8.9569312e-01 3.2832813e-01 9.9461603e-01 5.3082913e-02 9.9806857e-01]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [23]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.89      0.83        27
           1       0.86      0.72      0.78        25

    accuracy                           0.81        52
   macro avg       0.82      0.80      0.81        52
weighted avg       0.81      0.81      0.81        52



**You can see that by using dropout layer test accuracy increased from 0.77 to 0.81**